In [1]:
import pandas as pd
import numpy
import os
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

In [3]:
# seed
seed = 0
numpy.random.seed(3)
tf.random.set_seed(3)

In [4]:
df_pre = pd.read_csv('./dataset/wine.csv')
df = df_pre.sample(frac = 0.8)

In [5]:
dataset = df.values
X = dataset[:, 0:12]
Y = dataset[:, 12]

In [6]:
model = Sequential()
model.add(Dense(30, input_dim = 12, activation = 'relu'))
model.add(Dense(12, activation = 'relu'))
model.add(Dense(8, activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid'))

In [7]:
model.compile(loss = 'binary_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [8]:
# 학습 셋과 테스트 셋의 구분
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [9]:
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [10]:
modelpath = './model/{epoch:02d}-{val_loss:.4f}.hdf5'
checkpointer = ModelCheckpoint(filepath = modelpath, monitor = 'val_loss', verbose = 1,
                              save_best_only = True)

In [11]:
# 10개의 파일로 쪼갬
n_fold = 4
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [12]:
accuracy = []

In [13]:
no = 0
# 모델의 설정, 컴파일, 실행
for train, test in skf.split(X, Y):
    model = Sequential()
    model.add(Dense(30, input_dim = 12, activation = 'relu'))
    model.add(Dense(12, activation = 'relu'))
    model.add(Dense(8, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    model.fit(X[train], Y[train], epochs=100, batch_size=500, verbose=0)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)
    
    no = no + 1
    model.save(f'./model{no}.h5')

1299/1299 [==============================] - 0s 96us/sample - loss: 0.0777 - accuracy: 0.9738


In [14]:
# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)


 4 fold accuracy: ['0.9692', '0.9731', '0.9731', '0.9738']
